This is an example notebook file for tests where we balance a transistor against another transistor.
Note: In general, you should not run the code included here as written and expect everything to work. You will have to do some independent testing to get all of the relevant parameters right and tailor the algorithm to your system.

In [ ]:
"""IMPORTS"""
from pyPulses.devices import ad5764, mso44, keithley2400, pulseGenerator, watdScope
from pyPulses.utils import clearLoggers, getQuickLogger, tandemSweep
from pyPulses import DeviceRegistry

from pyPulses.utils import ExtrapPred1d, ExtrapPredNd, BalanceConfig, BrentSolver
from pyPulses.utils import balance1d, RootFinderState, RootFinderStatus

import numpy as np
import matplotlib.pyplot as plt
import logging
import time
import json
import os

import importlib

from scipy.interpolate import RegularGridInterpolator
from matplotlib import colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

logging_folder = #INSERT LOGGING FOLDER HERE
test_path = #INSERT TEST PATH HERE

In [ ]:
"""Pulsed measurement setup"""
"""Open all of the relevant instruments and set their default parameters."""

default_wait = 0.05
default_max_step = 0.1

# set up the DC box
ad5764_logger = getQuickLogger("ad5764", logging_folder)
dcbox = ad5764(ad5764_logger)
dcbox.wait = default_wait
dcbox.max_step = default_max_step
for ch in range(8): dcbox.set_V(ch, 0.0)

# set up the keithley, used for VDD on the amplifier
keithley2400_logger = getQuickLogger("keithley2400", logging_folder)
VDD = keithley2400(keithley2400_logger, instrument_id="GPIB0::18::INSTR")
VDD.wait = default_wait
VDD.max_step = default_max_step

# set up the pulse generator, disabling discharge pulses
pulseGenerator_logger = getQuickLogger("pulseGenerator", logging_folder)
pulse_gen = pulseGenerator(pulseGenerator_logger)
pulse_gen.dcbox_map = {
    "Vx1" : 6,
    "Vy1" : 4,
    "Vx2" : 7,
    "Vy2" : 5 
}
pulse_gen.set_V("Vx2", 0.0)
pulse_gen.set_V("Vy2", 0.0)
pulse_gen.dis_on(False)

# set up the oscilloscope as a time-domain waveform averager
mso44_logger = getQuickLogger("mso44", logging_folder)
watdScope_logger = getQuickLogger("watdScope", logging_folder)
watd = watdScope(loggers = (watdScope_logger, mso44_logger))

# Configure the scope
watd.tint0 = 110e-9
watd.tint1 = 280e-9
watd.trace_wait = 1.0
watd.scope.get_waveform_parameters()

"""DC measurement setup"""
Vb_logger = getQuickLogger("Vbias", logging_folder)
VBIAS = keithley2400(Vb_logger, instrument_id = "GPIB0::7::INSTR")

"""Voltage Protected Setters and other Functionality"""

# use this to set the amplifier drain voltage
def set_VDD(V): 
    if not 0 <= V <= 9.0:
        raise RuntimeError("VDD must be between 0 and 9 V")
    VDD.sweep_V(V)

# use this to set the amplifier gate voltage
_VGamp = 1
def set_VGamp(V): 
    if not -1.5 <= V <= 0:
        raise RuntimeError("VG must be between -1.5 and 0 V")
    dcbox.sweep_V(_VGamp, V)

def get_VGamp():
    return dcbox.get_V(_VGamp)

# use this to set the "device" gate voltage
_VGdev = 2
def set_VGdev(V): 
    if not -1.5 <= V <= 0:
        raise RuntimeError("VGb must be between -1.5 and 0 V")
    dcbox.sweep_V(_VGdev, V)

def get_VGdev():
    return dcbox.get_V(_VGdev)

# use this to set the "reference" gate voltage
_VGref = 3
def set_VGref(V):
    if not -1.5 <= V <= 0:
        raise RuntimeError("VG2 must be between -1.5 and 0 V")
    dcbox.sweep_V(_VGref, V)

def get_VGref():
    return dcbox.get_V(_VGref)

"""Function to sweep everything down"""
def sweep_down():
    VBIAS.set_V(0.0)
    pulse_gen.set_V("Vx1", 0.0)
    pulse_gen.set_V("Vy1", 0.0)
    pulse_gen.set_V("Vx2", 0.0)
    pulse_gen.set_V("Vy2", 0.0)
    set_VDD(0.0)
    set_VGamp(0.0)
    set_VGdev(0.0)
    set_VGref(0.0)

    # For good measure, sweep all of the DC box channels down
    tandemSweep(default_wait, 
        *[
            (lambda x: dcbox.set_V(ch, x), dcbox.get_V(ch), 0., default_max_step)
            for ch in range(8)
        ]
    )
    # For extra, extra security, set them all 0 after just to be sure
    for ch in range(8): dcbox.set_V(ch, 0.0)

In [ ]:
"""Take a map of I-V curves over gate space. (Sweeping Gate, Stepping Vx, for worst case scenario)"""

# Sparse gate rebalancing -- we only rebalance the gate when it is otherwise impossible to find a balance point

# Define the space over which we take the map
VGdev_npoints = 50
VX1_npoints = 50
VGdev_space = np.linspace(-0.0, -1.0, VGdev_npoints+1)[1:]
VX1_space = np.linspace(0., 4.0, VX1_npoints+1)[1:]

# Set up the outputs
VGref_balance1d_logger  = getQuickLogger("VGref_balance1d", logging_folder)
VY1_balance1d_logger    = getQuickLogger("VY1_balance1d", logging_folder)
metadata_path               = os.path.join(test_path, "map_metadata.json")
VGref_balance_points_path   = os.path.join(test_path, "VGref_balance_points.npy")
VY1_balance_points_path     = os.path.join(test_path, "VY1_balance_points.npy")

# More miscellaneous parameters
background = 3.409537916666668e-05
watd.tint0 = 50e-9
watd.tint1 = 110e-9
watd.trace_wait = 2.0
excitation_time = 100e-9
Vdd_setting = 8.0
Vg_amp_setting = -0.5

min_VGref, max_VGref = -1.2, -0.1
min_VY1, max_VY1 = 0.1, 4.5

# Set up the metadata file
metadata = {
    "VGdev_npoints"     : VGdev_npoints,
    "VX1_npoints"       : VX1_npoints,
    "VGdev_space"       : [VGdev_space[0], VGdev_space[-1], 
                           VGdev_space[1] - VGdev_space[0]],
    "VX1_space"         : [VX1_space[0], VX1_space[-1],
                           VX1_space[1] - VX1_space[0]],
    "VGref_path"        : VGref_balance_points_path,
    "VY1_path"          : VY1_balance_points_path,
    "skipped_points"    : [],
    "skipped_points_ind": [],
    "background"        : background,
    "mean_window"       : [watd.tint0, watd.tint1],
    "excitation_time"   : excitation_time,
    "Vdd_setting"       : Vdd_setting,
    "Vg_amp_setting"    : Vg_amp_setting,
    "VG_ref_range"      : [min_VGref, max_VGref],
    "VY1_range"         : [min_VY1, max_VY1],
}

with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent = 4)

# Precondition all of the levels
sweep_down()
VDD.sweep_V(Vdd_setting)
set_VGamp(Vg_amp_setting)
set_VGref(-0.2)
set_VGdev(0.0)
pulse_gen.set_V("Vx1", 0.0)
pulse_gen.set_V("Vy1", 0.0)

# Set up pulse widths and turn off discharge pulses
pulse_gen.set_polarity(True)
pulse_gen.set_exc_width(excitation_time)
pulse_gen.dis_on(False)

# Configure the scope
watd.scope.get_waveform_parameters()
def mean_window():
    watd.scope.clear_trace()
    watd.run(True)
    m = watd.take_mean()
    watd.run(False)
    return m - background

# Set up the gate balancing procedure
VGref_predictor = ExtrapPredNd(
    pspace_shape    = (VX1_npoints, VGdev_npoints),
    support         = (5, 5),
    order           = (1, 1),
    default0        = lambda p, *args: 0.9*p[1],
    default1        = lambda p, *args: 1.1*p[1],
    axes            = (VX1_space, VGdev_space)
)

VGref_balance_config = BalanceConfig(
    set_x           = set_VGref,
    get_y           = mean_window,
    predictor       = VGref_predictor,
    rootfinder      = BrentSolver,
    x_tolerance     = 1e-3,
    y_tolerance     = 1.5e-5,
    search_range    = (min_VGref, max_VGref),
    max_iter        = 20,
    max_reps        = 3,
    max_coll        = 5,
    logger          = VGref_balance1d_logger
)

# Set up the pulse height balancing procedure
VY1_predictor = ExtrapPredNd(
    pspace_shape    = (VX1_npoints, VGdev_npoints),
    support         = (5, 5),
    order           = (1, 1),
    default0        = lambda p, *args: 0.9*p[0],
    default1        = lambda p, *args: 1.1*p[0],
    axes            = (VX1_space, VGdev_space)
)

VY1_balance_config = BalanceConfig(
    set_x           = lambda x: pulse_gen.set_V("Vy1", x),
    get_y           = mean_window,
    predictor       = VY1_predictor,
    rootfinder      = BrentSolver,
    x_tolerance     = 1e-4,
    y_tolerance     = 1e-5,
    search_range    = (min_VY1, max_VY1),
    max_iter        = 20,
    max_reps        = 2,
    max_coll        = 2,
    logger          = VY1_balance1d_logger
)
    
for i, VX1 in enumerate(VX1_space):
    print("#"*50)
    print(f"##### MOVING TO NEW VX1 AMPLITUDE: {VX1:.5f} V #####")
    print("#"*50)

    dyn_min_VGref, dyn_max_VGref = min_VGref, max_VGref
    dyn_min_VY1, dyn_max_VY1 = max(min_VY1, 0.1*VX1), max_VY1
    VGref_balance_config.search_range = (dyn_min_VGref, dyn_max_VGref)
    VY1_balance_config.search_range = (dyn_min_VY1, dyn_max_VY1)

    pulse_gen.set_V("Vx1", VX1)

    for j, VG_dev in enumerate(VGdev_space):
        print("="*50)

        prog = (i*VX1_npoints + j)/(VGdev_npoints*VX1_npoints)*100
        print(f"Vx1: {VX1:.4f} V, Device Gate: {VG_dev:.4f} V ({prog:.2f}%)")

        set_VGdev(VG_dev)

        # Move VGref and VY1 to the new predicted balance point
        VY1_guess = VY1_predictor.predict0((VX1, VG_dev))
        VGref_guess = VGref_predictor.predict0((VX1, VG_dev))

        print(f"Predicted VY1   = {VY1_guess:.4f} V")
        print(f"Predicted VGref = {VGref_guess:.4f} V")

        if (not min_VY1 <= VY1_guess <= max_VY1) or \
            (not min_VGref <= VGref_guess <= max_VGref):
            print("Predicted balance point is out of bounds; Truncating")
            VY1_guess = max(min(dyn_max_VY1, VY1_guess), dyn_min_VY1)
            VGref_guess = max(min(dyn_max_VGref, VGref_guess), dyn_min_VGref)
            print(f"    New Predicted VY1   = {VY1_guess:.4f} V")
            print(f"    New Predicted VGref = {VGref_guess:.4f} V")

        pulse_gen.set_V("Vy1", VY1_guess)
        set_VGref(VGref_guess)

        print("Attempting to balance using pulse heights alone.")
        VY1_balance_state = balance1d((VX1, VG_dev), VY1_balance_config)

        print("RESULT:")
        print(VY1_balance_state)

        if VY1_balance_state.status == RootFinderStatus.CONVERGED:
            print(f"Successfully balanced using pulse heights.")
            print(f"    VGdev   = {VG_dev:.5f}")
            print(f"    VGref   = {get_VGref():.5f}")
            print(f"    VX1     = {VX1:.5f}")
            print(f"    VY1     = {VY1_balance_state.root:.5f}")

            # Write all of the results to their respective files
            VGref_predictor.update((VX1, VG_dev), get_VGref())
            VY1_predictor.update((VX1, VG_dev), VY1_balance_state.root)

            np.save(VGref_balance_points_path, VGref_predictor.balance_history)
            np.save(VY1_balance_points_path, VY1_predictor.balance_history)

        else:
            print("Unable to find balance with current VGref")

            VYt = max(min(dyn_max_VY1, VX1), dyn_min_VY1)
            print(f"Sweeping VY1 back to {VYt}")
            pulse_gen.set_V("Vy1", VYt)

            print("Attempting to rebalance the reference gate")

            VGref_balance_state = balance1d((VX1, VG_dev), VGref_balance_config)

            print("RESULT:")
            print(VGref_balance_state)

            if not VGref_balance_state.status == RootFinderStatus.CONVERGED:
                print("Failed to rebalance the gate")
                metadata["skipped_points"].append((VX1, VG_dev))
                metadata["skipped_points_ind"].append((i, j))
                with open(metadata_path, 'w') as meta_f:
                    json.dump(metadata, meta_f, indent = 4)
                continue

            print("Successfully rebalanced the gate")

            # Depending on how good our balance point is, refine using pulse heights

            if VGref_balance_state.best_value < VY1_balance_config.y_tolerance:
                print("Tolerance after rebalancing was sufficient")
                print("Forego fine tuning of VY1")

                VY1_balance_state = RootFinderState(
                    status      = RootFinderStatus.CONVERGED,
                    point       = VYt,
                    root        = VYt,
                    iterations  = 0,
                    message     = "Converged without having to fine tune",
                    best_value  = VGref_balance_state.best_value
                )
            else:
                print("Attempting to balance using pulse heights")
                VY1_balance_state = balance1d((VX1, VG_dev), VY1_balance_config)

            print("RESULT:")
            print(VY1_balance_state)

            if not VY1_balance_state.status == RootFinderStatus.CONVERGED:
                print("Failed to balance.")
                metadata["skipped_points"].append((VX1, VG_dev))
                metadata["skipped_points_ind"].append((i, j))
                with open(metadata_path, 'w') as meta_f:
                    json.dump(metadata, meta_f, indent = 4)
                continue

            print(f"Successfully balanced using pulse heights.")
            print(f"    VGdev   = {VG_dev:.5f}")
            print(f"    VGref   = {VGref_balance_state.root:.5f}")
            print(f"    VX1     = {VX1:.5f}")
            print(f"    VY1     = {VY1_balance_state.root:.5f}")

            # Write all of the results to their respective files
            VGref_predictor.update((VX1, VG_dev), get_VGref())
            VY1_predictor.update((VX1, VG_dev), VY1_balance_state.root)

            np.save(VGref_balance_points_path, VGref_predictor.balance_history)
            np.save(VY1_balance_points_path, VY1_predictor.balance_history)


    print(f"Finished sweep over VGdev at VX1 = {VX1:.5f} ({i+1}/{VX1_npoints})")
    print("Clearing WATD memory...")
    watd.scope.clear_trace()
    time.sleep(2.0)

print("#"*50)
print("#"*50)
print("")
print("FINISHED TAKING MAP. SWEEPING EVERYTHING DOWN")
sweep_down()
